In [1]:
from src import run_exp, statistics, get_graph, split_pos_neg
import networkx as nx
import pandas as pd
from pytorch_lightning import seed_everything
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [ ]:
baseline_methods = ['metis', 'louvain', 'eva', 'maxflow_cpp_unweighted', 'maxflow_cpp_weighted', 'gnn', 'pads_cpp']
datasets = ['Abortion', 'Brexit', 'Election', 'Gun', 'Partisanship', 'Referendum_']
seed_everything(20010302)

Seed set to 20010302


20010302

In [ ]:
for d in datasets[1:]:
    print(f"===Dataset {d}===")
    G = get_graph(d)
    split_pos_neg(G, f'input/datasets/static/{d}', 500)
    timer = {}
    for method in baseline_methods:
        timer[method], rst = run_exp(G, method, dataset=d, theta=0.1)
        print(f"{method} Finished! Time taken: {timer[method]}")
    nx.write_gml(G, f'output/results/{d}/graph.gml')
    pd.DataFrame(timer.items(), columns=['method', 'time']).to_csv(f'output/results/{d}/time.csv')

    df_pos, df_neg = statistics(G, d, f'output/results/{d}')
    print(df_pos)
    print(df_neg)

===Dataset Brexit===
metis Finished! Time taken: 0.094
louvain Finished! Time taken: 1.935
eva Finished! Time taken: 149.521
maxflow_cpp_unweighted Finished! Time taken: 0.603
maxflow_cpp_weighted Finished! Time taken: 0.508


Computing transition probabilities:   0%|          | 0/7589 [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

gnn Finished! Time taken: 347.302
pads_cpp Finished! Time taken: 0.013522
             method  num_nodes  num_edges purity(var) conductance  \
0           cascade       4423    63853.5     0.40819     0.07499   
1             metis       3795    60763.0     0.33664     0.05003   
2           louvain       3357    56147.0     0.30873     0.07137   
3               eva       3198    55745.5     0.28266     0.07449   
4  maxflow_cpp_udsp        403    18263.0     0.04081     0.49413   
5  maxflow_cpp_wdsp        389    17207.0     0.03291     0.50850   
6      node2vec_gin        396    18562.0     0.05700     0.49279   
7          pads_cpp        392    17528.5     0.03574     0.50541   

  avg_node_polarity unweighted_density weighted_density  
0           0.47996           14.43669          7.84296  
1           0.54335           16.01133          8.87396  
2           0.56512           16.72535          9.37779  
3           0.57136           17.43136          9.79630  
4           0.